In [1]:
import pandas as pd
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
eval_data = pd.read_csv("/content/drive/MyDrive/dataset/상담대화_평가용.csv")
eval_data = eval_data.drop('Unnamed: 0',axis=1)
eval_data = eval_data[:300]

eval_data = eval_data[~eval_data['text'].str.contains("안녕하세요")]
eval_data = eval_data.reset_index(drop=True)
len(eval_data)

270

In [3]:
speaker_data = eval_data[eval_data['speaker']==0].reset_index(drop=True) #생성모델의 input
listener_data = eval_data[eval_data['speaker']==1].reset_index(drop=True) #분류모델의 input

In [4]:
print(len(speaker_data))
print(len(listener_data))

136
134


In [5]:
min_length = min(len(speaker_data),len(listener_data))

df = pd.DataFrame({
    'speaker':speaker_data['text'][:min_length],
    'listener':listener_data['text'][:min_length]
})
df.head(10)

,speaker,listener
0,내가 약간 중2병 같은 걸 증상을 보이고 있어요.,"중2병 증상이라니, 어떤 증상이신 건가요?"
1,"그러니까 공부하기 싫어하고, 공격적이고, 좀 무례하게 말하고 싶은 게 많아져서 그런...",그런 증상이 있으니까 힘드시겠죠. 중2병 같은 것이라고 생각하시는 이유는 무엇인가요?
2,막 공부 안하고 이것저것 들먹이고 하고 싶은 게 너무 많아서 그런 거 같아요.,그런 것도 어쩔 수 없이 찾아오는 시기가 있으니까 무리하지 않도록 해야겠죠. 대학교...
3,그걸 제가 잘 몰라서 그런 것 같아요. 그냥 더 심해졌다고 느꼈어요.,대학교 생활이 신나고 재밌으신 건 어떤 점이 있나요?
4,학과가 정말 좋아서 즐겁게 수업을 듣고 있어요. 학우들도 좋고 괜찮은 친구들도 많이...,즐거운 일도 많이 있으면서 고민거리도 있는 것 같군요. 가사나 소설을 쓰시면서 마음...
5,"좋은 질문이에요. 좀 자세히 말씀드릴게요. 학교에서 어려운 일이 있었는데, 그 때부...","어려운 일이 있으셨군요. 그 때부터 쓰시면서 나아지는 기분이 드셨다고 하셨는데, 현..."
6,"좀 그래요. 이제는 어느 정도 극복했다고 생각은 하지만, 가사나 소설을 쓰면서 마음...","가사나 소설 같은 것들이 나쁜 것은 아니라고 하셨죠. 하지만, 과도한 습관으로 만들..."
7,"그럼, 가사나 소설 같은 것들을 과도하게 하면 나중에 문제가 생긴다는 건가요?",그렇습니다. 이러한 습관이 과도하게 되면 자신에게 과부하를 주고 심리적인 문제를 일...
8,그래도 좀 걱정이 되네요.,걱정은 당연한 것이죠. 그러나 지나치게 걱정하는 것은 오히려 스트레스를 더 받게 됩...
9,그 말씀대로 할게요.,그렇게 하셔서 조금이나마 좋아지시길 바라겠습니다. 이후에도 힘든 마음이 계속되면 언...


In [6]:
! pip install peft

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [7]:
import torch
from peft import LoraConfig, TaskType, get_peft_model
from transformers import PreTrainedTokenizerFast, AutoModelForSequenceClassification, GPT2LMHeadModel

- lora 적용

In [8]:
cls_peft_config = LoraConfig(
    task_type="SEQ_CLS",
    inference_mode=True,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none")

gen_peft_config = LoraConfig(
    task_type="CAUSAL_LM",
    inference_mode=True,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none")

- 인자

In [9]:
# 변수
# 그 외 추가 가능 인자 https://huggingface.co/docs/transformers/en/main_classes/text_generation#transformers.GenerationConfig

threshold=0.55
max_length = 45
min_new_tokens = 3
use_cache = True
repetition_penalty = 4.0
do_sample = False
num_beams = 2
temperature = 2.0
top_k = 50
top_p = 1.0

- 토큰

In [10]:
Q_TKN = ""
A_TKN = ""
BOS = ''
EOS = ''
UNK = ''
MASK = ''
SENT = ''
PAD = ''

1. 분류 모델로 true_emotion 구하기

In [11]:
# classfication model load
cls_path = '/content/drive/MyDrive/kogpt2-chatbot'
model = AutoModelForSequenceClassification.from_pretrained(
      cls_path,
      num_labels=5,
      problem_type="multi_label_classification"
)
cls_model = get_peft_model(model, cls_peft_config)

cls_tokenizer = PreTrainedTokenizerFast.from_pretrained(cls_path)
                          #bos_token=BOS, eos_token=EOS, unk_token=UNK,
                          #pad_token=PAD, mask_token=MASK)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at skt/kogpt2-base-v2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
def emotion_classification(input_text, model, tokenizer, threshold=threshold):
    model.eval()

    # 입력 문장 토큰화
    inputs = tokenizer(input_text, return_tensors='pt', truncation=True, padding=True, max_length=128)

    # 모델에 입력을 전달하여 로짓(logits)을 얻음
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits

    # 로짓에 시그모이드 적용하여 확률로 변환
    probabilities = torch.sigmoid(logits)
    # 임계값을 기준으로 이진화
    predictions = (probabilities > threshold).int()

    # 레이블 디코딩
    label_classes = ['조언', '격려', '위로', '동조', '']
    num_classes = 5
    predicted_labels = [label_classes[i] for i in range(num_classes) if predictions[0][i] == 1]

    return predicted_labels

In [13]:
df['real_label'] = df['listener'].apply(lambda x: emotion_classification(x, cls_model, cls_tokenizer))

In [14]:
df.head()

,speaker,listener,real_label
0,내가 약간 중2병 같은 걸 증상을 보이고 있어요.,"중2병 증상이라니, 어떤 증상이신 건가요?",[]
1,"그러니까 공부하기 싫어하고, 공격적이고, 좀 무례하게 말하고 싶은 게 많아져서 그런...",그런 증상이 있으니까 힘드시겠죠. 중2병 같은 것이라고 생각하시는 이유는 무엇인가요?,[]
2,막 공부 안하고 이것저것 들먹이고 하고 싶은 게 너무 많아서 그런 거 같아요.,그런 것도 어쩔 수 없이 찾아오는 시기가 있으니까 무리하지 않도록 해야겠죠. 대학교...,[]
3,그걸 제가 잘 몰라서 그런 것 같아요. 그냥 더 심해졌다고 느꼈어요.,대학교 생활이 신나고 재밌으신 건 어떤 점이 있나요?,[]
4,학과가 정말 좋아서 즐겁게 수업을 듣고 있어요. 학우들도 좋고 괜찮은 친구들도 많이...,즐거운 일도 많이 있으면서 고민거리도 있는 것 같군요. 가사나 소설을 쓰시면서 마음...,[]


In [15]:
df['real_label'] = df['real_label'].apply(lambda x: str(x) if isinstance(x, list) else x)
unique_labels = df['real_label'].unique()
print(unique_labels)

['[]' "['']" "['동조']" "['위로', '동조']" "['위로', '']" "['조언', '동조']"
 "['격려', '위로', '동조']" "['격려', '동조']" "['동조', '']" "['조언']"
 "['격려', '동조', '']" "['격려', '']" "['격려']" "['위로']" "['조언', '위로', '동조']"]


2. 생성 모델 답변 생성

In [16]:
# gen model load
gen_path = '/content/drive/MyDrive/kogpt2-chatbot'
gen_model = GPT2LMHeadModel.from_pretrained(gen_path)

gen_model = get_peft_model(gen_model, gen_peft_config)
gen_tokenizer = PreTrainedTokenizerFast.from_pretrained(gen_path,
                          bos_token=BOS, eos_token=EOS, unk_token=UNK,
                          pad_token=PAD, mask_token=MASK)

In [27]:
threshold=0.55
max_length = 100
min_new_tokens = 3
use_cache = True
repetition_penalty = 4.0
do_sample = False
num_beams = 2
temperature = 2.0
top_k = 50
top_p = 1.0

In [28]:
def predict_answer(predicted_labels, input_text, model, tokenizer,
                   max_length, min_new_tokens, use_cache,
                   repetition_penalty, do_sample, num_beams,
                   temperature, top_k, top_p):
    # 모델을 평가 모드로 전환
    model.eval()
    # 입력 문장 토큰화
    empathy = ' ,'.join(map(str, predicted_labels))
    inputs = Q_TKN + input_text + SENT + empathy + A_TKN
    input_ids = tokenizer.encode(tokenizer.bos_token + inputs + tokenizer.eos_token, return_tensors='pt')

    # 모델 추론
    outputs = model.generate(input_ids,
                             max_length=max_length, min_new_tokens=min_new_tokens, use_cache=use_cache,
                             repetition_penalty=repetition_penalty, do_sample=do_sample, num_beams=num_beams,
                             temperature=temperature, top_k=top_k, top_p=top_p, early_stopping=True)
    output_text = gen_tokenizer.decode(outputs[0], skip_special_tokens=True)
    output_text = output_text.split(A_TKN)[1]

    return output_text

In [ ]:
# 예외 처리 추가
def apply_generate_response(row):
    try:
        return predict_answer(row['real_label'], row['speaker'], gen_model, gen_tokenizer,max_length, min_new_tokens, use_cache,
                   repetition_penalty, do_sample, num_beams,
                   temperature, top_k, top_p)
    except Exception as e:
        print(f"Error generating response for row {row.name}: {e}")
        return None

# 응답 생성 및 저장
df['generated_response'] = df.apply(apply_generate_response, axis=1)

Error generating response for row 0: empty separator
Error generating response for row 1: empty separator
Error generating response for row 2: empty separator
Error generating response for row 3: empty separator
Error generating response for row 4: empty separator
Error generating response for row 5: empty separator
Error generating response for row 6: empty separator
Error generating response for row 7: empty separator
Error generating response for row 8: empty separator
Error generating response for row 9: empty separator
Error generating response for row 10: empty separator
Error generating response for row 11: empty separator
Error generating response for row 12: empty separator
Error generating response for row 13: empty separator
Error generating response for row 14: empty separator
Error generating response for row 15: empty separator
Error generating response for row 16: empty separator
Error generating response for row 17: empty separator
Error generating response for row 18: 

In [19]:
'''def generate_response(predicted_labels, input_text, model, tokenizer,
                      max_length=100, min_new_tokens=2, use_cache=True,
                      repetition_penalty=5.0, do_sample=True, num_beams=5,
                      temperature=2.0, top_k=50, top_p=0.9, max_new_tokens=50):
    # 모델을 평가 모드로 전환
    model.eval()
    # 입력 문장 토큰화
    empathy = ','.join(map(str, predicted_labels))
    inputs = Q_TKN + input_text + SENT + empathy + A_TKN
    input_ids = tokenizer.encode(BOS + inputs + EOS, return_tensors='pt')

    # 입력 토큰 길이 확인 및 조정
    if input_ids.size(1) > max_length - max_new_tokens:
        input_ids = input_ids[:, -(max_length - max_new_tokens):]

    # 모델 추론
    outputs = model.generate(input_ids,
                             max_length=max_length, min_new_tokens=min_new_tokens, use_cache=use_cache,
                             repetition_penalty=repetition_penalty, do_sample=do_sample, num_beams=num_beams,
                             temperature=temperature, top_k=top_k, top_p=top_p, early_stopping=True,
                             max_new_tokens=max_new_tokens)
    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # A_TKN을 기준으로 분할하여 응답 부분만 추출
    if A_TKN and A_TKN in output_text:
        output_text = output_text.split(A_TKN)[1].strip()
    else:
        output_text = output_text.strip()

    return output_text

# 예외 처리 추가
def apply_generate_response(row):
    try:
        return generate_response(row['real_label'], row['speaker'], gen_model, gen_tokenizer)
    except Exception as e:
        print(f"Error generating response for row {row.name}: {e}")
        return None

# 응답 생성 및 저장
df['generated_response'] = df.apply(apply_generate_response, axis=1)

# 결과 출력
print(df[['speaker', 'listener', 'generated_response']])'''

'def generate_response(predicted_labels, input_text, model, tokenizer,\n                      max_length=100, min_new_tokens=2, use_cache=True,\n                      repetition_penalty=5.0, do_sample=True, num_beams=5,\n                      temperature=2.0, top_k=50, top_p=0.9, max_new_tokens=50):\n    # 모델을 평가 모드로 전환\n    model.eval()\n    # 입력 문장 토큰화\n    empathy = \',\'.join(map(str, predicted_labels))\n    inputs = Q_TKN + input_text + SENT + empathy + A_TKN\n    input_ids = tokenizer.encode(BOS + inputs + EOS, return_tensors=\'pt\')\n\n    # 입력 토큰 길이 확인 및 조정\n    if input_ids.size(1) > max_length - max_new_tokens:\n        input_ids = input_ids[:, -(max_length - max_new_tokens):]\n\n    # 모델 추론\n    outputs = model.generate(input_ids,\n                             max_length=max_length, min_new_tokens=min_new_tokens, use_cache=use_cache,\n                             repetition_penalty=repetition_penalty, do_sample=do_sample, num_beams=num_beams,\n                             te

In [20]:
df.head(10)

,speaker,listener,real_label,generated_response
0,내가 약간 중2병 같은 걸 증상을 보이고 있어요.,"중2병 증상이라니, 어떤 증상이신 건가요?",[],None
1,"그러니까 공부하기 싫어하고, 공격적이고, 좀 무례하게 말하고 싶은 게 많아져서 그런...",그런 증상이 있으니까 힘드시겠죠. 중2병 같은 것이라고 생각하시는 이유는 무엇인가요?,[''],None
2,막 공부 안하고 이것저것 들먹이고 하고 싶은 게 너무 많아서 그런 거 같아요.,그런 것도 어쩔 수 없이 찾아오는 시기가 있으니까 무리하지 않도록 해야겠죠. 대학교...,[],None
3,그걸 제가 잘 몰라서 그런 것 같아요. 그냥 더 심해졌다고 느꼈어요.,대학교 생활이 신나고 재밌으신 건 어떤 점이 있나요?,[],None
4,학과가 정말 좋아서 즐겁게 수업을 듣고 있어요. 학우들도 좋고 괜찮은 친구들도 많이...,즐거운 일도 많이 있으면서 고민거리도 있는 것 같군요. 가사나 소설을 쓰시면서 마음...,[],None
5,"좋은 질문이에요. 좀 자세히 말씀드릴게요. 학교에서 어려운 일이 있었는데, 그 때부...","어려운 일이 있으셨군요. 그 때부터 쓰시면서 나아지는 기분이 드셨다고 하셨는데, 현...",['동조'],None
6,"좀 그래요. 이제는 어느 정도 극복했다고 생각은 하지만, 가사나 소설을 쓰면서 마음...","가사나 소설 같은 것들이 나쁜 것은 아니라고 하셨죠. 하지만, 과도한 습관으로 만들...",[],None
7,"그럼, 가사나 소설 같은 것들을 과도하게 하면 나중에 문제가 생긴다는 건가요?",그렇습니다. 이러한 습관이 과도하게 되면 자신에게 과부하를 주고 심리적인 문제를 일...,"['위로', '동조']",None
8,그래도 좀 걱정이 되네요.,걱정은 당연한 것이죠. 그러나 지나치게 걱정하는 것은 오히려 스트레스를 더 받게 됩...,[],None
9,그 말씀대로 할게요.,그렇게 하셔서 조금이나마 좋아지시길 바라겠습니다. 이후에도 힘든 마음이 계속되면 언...,['동조'],None


3. 감정 분류 모델을 통해서 감정 예측하기

In [21]:
# classfication model load
cls_path = '/content/drive/MyDrive/kogpt2-chatbot'
model = AutoModelForSequenceClassification.from_pretrained(
      cls_path,
      num_labels=5,
      problem_type="multi_label_classification"
)
cls_model = get_peft_model(model, cls_peft_config)
cls_tokenizer = PreTrainedTokenizerFast.from_pretrained(cls_path,
                          bos_token=BOS, eos_token=EOS, unk_token=UNK,
                          pad_token=PAD, mask_token=MASK)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at skt/kogpt2-base-v2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
import torch

#감정 분류 추론
def emotion_classification(text, model, tokenizer, max_len=128):
    model.eval()

    # 토크나이징
    encoding = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=max_len,
        return_token_type_ids=False,
        padding='max_length',
        return_attention_mask=True,
        return_tensors='pt',
        truncation=True,
    )

    input_ids = encoding['input_ids']
    attention_mask = encoding['attention_mask']

    # 모델 예측
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits

    probs = torch.sigmoid(logits).cpu().numpy()[0]
    predictions = (probs > 0.5).astype(int)

    return predictions

In [23]:
#listenertext의 실제 감정 추론
true_emotions = []

#for i in range(len(listener_data)):
    #true_emotions.append(emotion_classification(listener_data.iloc[i]['text'], cls_model, cls_tokenizer, max_len=128))

true_emotions = [emotion_classification(df['listener'], cls_model, cls_tokenizer, max_len=128) for text in listener_data['text']]

RecursionError: maximum recursion depth exceeded

In [ ]:
#예측한 답변 감정 추론

pred_emotions = []

#generate_responses = listener_data['generate_response']
#pred_emotions = [emotion_classification(response) for response in generate_responses]

pred_emotions = [emotion_classification(df['generated_response'], cls_model, cls_tokenizer, max_len=128) for response in listener_data['generated_response']]

In [ ]:
from sklearn.metrics import classification_report
import numpy as np

#accuracy
print(classification_report(true_emotions, pred_emotions))

# MAE 계산
def calculate_mae(true_emotions, pred_emotions):
    true_emotions = np.array(true_emotions)
    pred_emotions = np.array(pred_emotions)
    mae = np.mean(np.abs(true_emotions - pred_emotions))
    return mae

mae = calculate_mae(true_emotions, pred_emotions)
print(f"Mean Absolute Error (MAE): {mae}")

---

In [ ]:
! pip install anthropic numpy scipy matplotlib

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [ ]:
import anthropic
import numpy as np
from scipy.spatial.distance import cosine
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd

In [ ]:
from voyageai import Client
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
!pip install -U voyageai

In [ ]:
client = Client(api_key="pa-hcwX_-1DRDb_5vmEThd36CnfwI5woxSGLWCtravhRZQ")

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/dataset/상담대화_평가용.csv")

In [ ]:
df.head()

In [ ]:
len(df)

In [ ]:
df['cosine_similarity'] = np.nan

In [ ]:
def get_similarity(text1, text2):
  embedding1 = client.embed(text1).embeddings
  embedding2 = client.embed(text2).embeddings

  embedding1_np = np.array(embedding1).flatten()
  embedding2_np = np.array(embedding2).flatten()
  similarity = cosine_similarity([embedding1_np], [embedding2_np])[0][0]
  return similarity

In [ ]:
for i in range(0, 299, 2):
  text1 = df.loc[i, 'text']
  text2 = df.loc[i+1, 'text']

  similarity = get_similarity(text1, text2)
  df.loc[i+1, 'cosine_similarity'] =similarity

In [ ]:
mean_similarity = df['cosine_similarity'].dropna().mean()
print(mean_similarity)

In [ ]:
#from sklearn.decomposition import PCA

#pca = PCA(n_components=3)
#embeddings_3d = pca.fit_transform(embeddings)

In [ ]:
#3d plot

#plt.rc('font', family='NanumBarunGothic')
#fig = plt.figure(figsize = (12,8))
#ax = fig.add_subplot(111, projection='3d')

#for i, (x,y,z) in enumerate(embeddings_3d):
#  ax.scatter(x,y,z)
#  ax.text(x,y,z, f'문장 {i+1}')


#ax.set_xlabel('X')
#ax.set_ylabel('Y')
#ax.set_zlabel('Z')
#ax.set_title('문장 임베딩의 3차원 표현')

#plt.show()